In [13]:
import matplotlib.pyplot as plt
import numpy as np
from main import *

In [14]:
# %matplotlib notebook
%matplotlib qt5

In [15]:
img = read_raw_simple("./nikon_D850.nef")
plt.imshow(img)

RAW Shape:(4032, 6032)
OB 600 
 white_level 15311 
 MAX2800


In [32]:
def impoly(img,poly_position=None): #四边形框选图像ROI
    "(rgb_mean,rgb_std,poly_position)=impoly(img)\n(rgb_mean,rgb_std,poly_position)=impoly(img,poly_position)"
    import matplotlib.pyplot as plt
    if poly_position is None:
        fig=plt.figure(figsize=[12.,7.5],tight_layout=True)
        h_img=plt.imshow(img)
        fig.show()
        # fig.canvas.set_window_title('waiting. ..')
        fig.canvas.manager.set_window_title('waiting. ..')
        pos=plt.ginput(n=4)
        # plt.close(fig)
        print(pos)
    else:
        pos=poly_position
    plt.waitforbuttonpress()
    (n,m)=np.meshgrid(np.arange(0.5,6.5)/6,np.arange(0.5,4.5)/4)
    n=n.flatten()
    m=m.flatten()
    x_center=(1-m)*((1-n)*pos[0][0]+n*pos[1][0])+m*(n*pos[2][0]+(1-n)*pos[3][0])
    y_center=(1-m)*((1-n)*pos[0][1]+n*pos[1][1])+m*(n*pos[2][1]+(1-n)*pos[3][1])
    r_sample=min([
        ((pos[0][0]-pos[1][0])**2+(pos[0][1]-pos[1][1])**2)**0.5/6,
        ((pos[1][0]-pos[2][0])**2+(pos[1][1]-pos[2][1])**2)**0.5/4,
        ((pos[2][0]-pos[3][0])**2+(pos[2][1]-pos[3][1])**2)**0.5/6,
        ((pos[3][0]-pos[0][0])**2+(pos[3][1]-pos[0][1])**2)**0.5/4,
        ])*0.2
    print(x_center)
    print(y_center)
    print(r_sample)
    if poly_position is None:
        plt.plot(pos[0][0],pos[0][1],'r+')
        plt.plot(pos[1][0],pos[1][1],'r+')
        plt.plot(pos[2][0],pos[2][1],'r+')
        plt.plot(pos[3][0],pos[3][1],'r+')
        # plt.plot(x_center,y_center,'yo')
        plt.plot(x_center-r_sample,y_center-r_sample,'y+')
        plt.plot(x_center+r_sample,y_center-r_sample,'y+')
        plt.plot(x_center-r_sample,y_center+r_sample,'y+')
        plt.plot(x_center+r_sample,y_center+r_sample,'y+')
        plt.draw()
        fig.show()
        poly_position=pos
    else:
        pass
    plt.waitforbuttonpress()
    rgb_mean=np.zeros((24,3))   
    rgb_std=np.zeros((24,3))   
    for block_idx in range(24):
        block=img[int(y_center[block_idx]-r_sample):int(y_center[block_idx]+r_sample),
                  int(x_center[block_idx]-r_sample):int(x_center[block_idx]+r_sample),:]
        rgb_vector,_=im2vector(block)
        rgb_mean[block_idx,:]=rgb_vector.mean(axis=0)
        rgb_std[block_idx,:]=rgb_vector.std(axis=0)
    # plt.close(fig)
    return (rgb_mean,rgb_std,poly_position,h_img,fig)

impoly(img)

[1024.25362684 1204.75439211 1385.25515739 1565.75592266 1746.25668793
 1926.7574532  1031.86000346 1211.64091038 1391.42181731 1571.20272423
 1750.98363116 1930.76453808 1039.46638007 1218.52742865 1397.58847723
 1576.6495258  1755.71057438 1934.77162296 1047.07275668 1225.41394691
 1403.75513715 1582.09632738 1760.43751761 1938.77870784]
[ 520.97013042  633.73836368  746.50659694  859.27483021  972.04306347
 1084.81129673  870.56631021  950.14247306 1029.71863591 1109.29479875
 1188.8709616  1268.44712445 1220.16249    1266.54658244 1312.93067487
 1359.3147673  1405.69885973 1452.08295217 1569.75866979 1582.95069181
 1596.14271383 1609.33473585 1622.52675787 1635.71877988]
33.415920772306784


(array([[0.0036941 , 0.00811451, 0.00647175],
        [0.01714766, 0.03225225, 0.02437791],
        [0.00094155, 0.00202971, 0.00165346],
        [0.00073005, 0.0015609 , 0.00130435],
        [0.00088813, 0.00185844, 0.00154085],
        [0.00736901, 0.02484988, 0.01776432],
        [0.00054592, 0.00114861, 0.00096129],
        [0.00082587, 0.00171238, 0.00138937],
        [0.00070979, 0.00146895, 0.00122028],
        [0.00457453, 0.01130214, 0.00447464],
        [0.01530443, 0.03417203, 0.03946025],
        [0.02641323, 0.03178373, 0.00647089],
        [0.00197884, 0.00297817, 0.00198189],
        [0.00263779, 0.0077767 , 0.01287733],
        [0.02050244, 0.01302933, 0.00907079],
        [0.00388341, 0.00614207, 0.00976085],
        [0.01272337, 0.01902569, 0.01194219],
        [0.00662888, 0.03326491, 0.03791673],
        [0.0082924 , 0.0181188 , 0.01505642],
        [0.01379793, 0.02968338, 0.02269753],
        [0.01565622, 0.03359412, 0.02564852],
        [0.01054426, 0.02214245, 0

In [39]:
def tellme(s):
    print(s)
    plt.title(s, fontsize=16)
    plt.draw()

def im2vector(img): #将图片转换为向量形式
    '''
        img [H,W,3]
        rgb [H*W,3]
        func_reverse reshape rgb to img
    '''
    size=img.shape
    rgb=np.reshape(img,(size[0]*size[1],3))
    func_reverse=lambda rgb : np.reshape(rgb,(size[0],size[1],size[2]))
    return rgb, func_reverse    

def manual_wb_area(img):
    '''
        img [H,W,3]

        list of selected wb area ([xmin,xmax,ymin,ymax],rgb_mean)
    '''
    plt.figure()
    plt.imshow(img)
    
    tellme("Select 2 points for a region to calculate WB, click to begin")
    plt.waitforbuttonpress()

    wb_list = []

    while True:
        pts = []
        while len(pts) < 2:
            tellme('Select 2 corners with mouse')
            pts = np.asarray(plt.ginput(2, timeout=-1))
            if len(pts) < 2:
                tellme('Too few points, starting over')
                time.sleep(1)  # Wait a second
        
        xmin = int(min(pts[:,0]))
        ymin = int(min(pts[:,1]))
        xmax = int(max(pts[:,0]))
        ymax = int(max(pts[:,1]))
        
        ph = plt.fill([xmin,xmax,xmax,xmin], [ymin,ymin,ymax,ymax], 'r', lw=2,alpha=0.3)
        
        block = img[xmin:xmax,ymin:ymax,:] #[H1,W1,3]
        rgb_vector,_=im2vector(block) #[H1*W1,3]
        rgb_mean=rgb_vector.mean(axis=0) #[3]
        rgb_std=rgb_vector.std(axis=0) #[3]
        
        print(f"Block at X:{xmin} {xmax} Y:{ymin} {ymax} Total Pixel {rgb_vector.shape[0]}")
        print(f"Mean {rgb_mean}")
        print(f"STD {rgb_std}")

        wb_list.append(([xmin,xmax,ymin,ymax],rgb_mean))

        tellme('Key click for quit, mouse click for extra data')
        if plt.waitforbuttonpress():
            break

        # # Get rid of fill
        # for p in ph:
        #     p.remove()

    return wb_list

manual_wb_area(img)


Select 2 points for a region to calculate WB, click to begin
Select 2 corners with mouse
Too few points, starting over
Select 2 corners with mouse
Block at X:1720 1816 Y:1276 1357 Total Pixel 7776
Mean [0.001096   0.00230908 0.00189048]
STD [0.00029311 0.0005414  0.00042384]
Key click for quit, mouse click for extra data
Select 2 corners with mouse
Block at X:1564 1639 Y:1439 1522 Total Pixel 6225
Mean [0.01652202 0.0350621  0.026484  ]
STD [0.01178881 0.02491412 0.01844034]
Key click for quit, mouse click for extra data
Select 2 corners with mouse
Block at X:1748 1792 Y:1451 1546 Total Pixel 4180
Mean [0.00188272 0.00412612 0.00339005]
STD [0.00243773 0.00538723 0.00411562]
Key click for quit, mouse click for extra data


[([1720, 1816, 1276, 1357],
  array([0.001096  , 0.00230908, 0.00189048], dtype=float32)),
 ([1564, 1639, 1439, 1522],
  array([0.01652202, 0.0350621 , 0.026484  ], dtype=float32)),
 ([1748, 1792, 1451, 1546],
  array([0.00188272, 0.00412612, 0.00339005], dtype=float32))]

In [27]:
import time

import numpy as np
import matplotlib.pyplot as plt


def tellme(s):
    print(s)
    plt.title(s, fontsize=16)
    plt.draw()

plt.figure()
plt.xlim(0, 1)
plt.ylim(0, 1)

tellme('You will define a triangle, click to begin')

print(plt.waitforbuttonpress())


while True:
    pts = []
    while len(pts) < 3:
        print(pts)
        tellme('Select 3 corners with mouse')
        pts = np.asarray(plt.ginput(3, timeout=-1))
        if len(pts) < 3:
            tellme('Too few points, starting over')
            time.sleep(1)  # Wait a second

    ph = plt.fill(pts[:, 0], pts[:, 1], 'r', lw=2)

    tellme('Happy? Key click for yes, mouse click for no')

    if plt.waitforbuttonpress():
        break

    # Get rid of fill
    for p in ph:
        p.remove()



You will define a triangle, click to begin
False
[]
Select 3 corners with mouse
Happy? Key click for yes, mouse click for no


In [ ]:
# Define a nice function of distance from individual pts
def f(x, y, pts):
    z = np.zeros_like(x)
    for p in pts:
        z = z + 1/(np.sqrt((x - p[0])**2 + (y - p[1])**2))
    return 1/z


X, Y = np.meshgrid(np.linspace(-1, 1, 51), np.linspace(-1, 1, 51))
Z = f(X, Y, pts)

CS = plt.contour(X, Y, Z, 20)

tellme('Use mouse to select contour label locations, middle button to finish')
CL = plt.clabel(CS, manual=True)

tellme('Now do a nested zoom, click to begin')
plt.waitforbuttonpress()

while True:
    tellme('Select two corners of zoom, middle mouse button to finish')
    pts = plt.ginput(2, timeout=-1)
    if len(pts) < 2:
        break
    (x0, y0), (x1, y1) = pts
    xmin, xmax = sorted([x0, x1])
    ymin, ymax = sorted([y0, y1])
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)

tellme('All Done!')
plt.show()

Use mouse to select contour label locations, middle button to finish
Select label locations manually using first mouse button.
End manual selection with second mouse button.
Now do a nested zoom, click to begin
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
All Done!


In [ ]:
plt.fill(pts[:, 0], pts[:, 1], 'b', lw=5)
plt.show()